The Dataset was obtained through [Kaggle](https://www.kaggle.com/datasets/ulrikthygepedersen/fastfood-nutrition/data). 

This notebook aims to clean and wrangle raw data for it to be ready to be analyzed.

# DATA CLEANING

In [1]:
import pandas as pd
import numpy as np

#set the working directory where the dataset lives
file_path = "C:/Users/Patrick Behnke/Desktop/Data Analytics/Projects/Project 2/Dataset/fastfood.csv"

df = pd.read_csv(file_path)


#quick look on data
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   restaurant   515 non-null    object 
 1   item         515 non-null    object 
 2   calories     515 non-null    int64  
 3   cal_fat      515 non-null    int64  
 4   total_fat    515 non-null    int64  
 5   sat_fat      515 non-null    float64
 6   trans_fat    515 non-null    float64
 7   cholesterol  515 non-null    int64  
 8   sodium       515 non-null    int64  
 9   total_carb   515 non-null    int64  
 10  fiber        503 non-null    float64
 11  sugar        515 non-null    int64  
 12  protein      514 non-null    float64
 13  vit_a        301 non-null    float64
 14  vit_c        305 non-null    float64
 15  calcium      305 non-null    float64
 16  salad        515 non-null    object 
dtypes: float64(7), int64(7), object(3)
memory usage: 68.5+ KB


#### DISCLAIMER: For analysis, rows missing fiber or protein values were dropped to ensure integrity of primary nutrient metrics. For micronutrients (Vitamin A, Vitamin C, Calcium), missing values were assumed to be zero (not present/negligible) to retain menu item diversity and avoid unnecessary row loss.

### Verify all the cells that has missing value is being treated as Null.

In [2]:
#make sure NaN/NA values are treated as null
df = df.replace({np.nan: None})

#check total missing values
df.isnull().sum()

restaurant       0
item             0
calories         0
cal_fat          0
total_fat        0
sat_fat          0
trans_fat        0
cholesterol      0
sodium           0
total_carb       0
fiber           12
sugar            0
protein          1
vit_a          214
vit_c          210
calcium        210
salad            0
dtype: int64

#### Null values from vitamins are accepted as deficiencies can occur from poor diet.

### Remove leading/trailing spaces, duplicates, and ensure data integrity.

In [3]:
#get the columns with restaurant names and items and remove the leading/trailing space
str_cols = ["restaurant", "item"]
df[str_cols] = df[str_cols].apply(lambda s: s.str.strip())

#remove the missing values from fiber and protein column
df_cleaned = df.dropna(subset = ["fiber", "protein"]).drop_duplicates(subset = ["item"])

#ensure string columns are string
df_cleaned[str_cols] = df_cleaned[str_cols].astype(str)

#ensure integer columns are float with 1 decimal place
num_cols = df_cleaned.columns[2:]
df_cleaned[num_cols] = df_cleaned[num_cols].apply(pd.to_numeric, errors="coerce").astype("float64") 

#convert NaN to 0 
nutrient_cols = ["vit_a", "vit_c", "calcium"]
df_cleaned[nutrient_cols] = df_cleaned[nutrient_cols].fillna(0)

df_cleaned.isnull().sum()

restaurant       0
item             0
calories         0
cal_fat          0
total_fat        0
sat_fat          0
trans_fat        0
cholesterol      0
sodium           0
total_carb       0
fiber            0
sugar            0
protein          0
vit_a            0
vit_c            0
calcium          0
salad          493
dtype: int64

### Remove unnecessary columns and create calculated columns for the analysis.

In [4]:
#salad column won't be used for this analysis
df_cleaned = df_cleaned.drop("salad", axis=1)

#get the protein efficiency (protein per calorie)
df_cleaned["protein_efficiency"] = df_cleaned["protein"] / df_cleaned["calories"] * 100

df_cleaned.head()

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,protein_efficiency
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380.0,60.0,7.0,2.0,0.0,95.0,1110.0,44.0,3.0,11.0,37.0,4.0,20.0,20.0,9.736842
1,Mcdonalds,Single Bacon Smokehouse Burger,840.0,410.0,45.0,17.0,1.5,130.0,1580.0,62.0,2.0,18.0,46.0,6.0,20.0,20.0,5.476190
2,Mcdonalds,Double Bacon Smokehouse Burger,1130.0,600.0,67.0,27.0,3.0,220.0,1920.0,63.0,3.0,18.0,70.0,10.0,20.0,50.0,6.194690
3,Mcdonalds,Grilled Bacon Smokehouse Chicken Sandwich,750.0,280.0,31.0,10.0,0.5,155.0,1940.0,62.0,2.0,18.0,55.0,6.0,25.0,20.0,7.333333
4,Mcdonalds,Crispy Bacon Smokehouse Chicken Sandwich,920.0,410.0,45.0,12.0,0.5,120.0,1980.0,81.0,4.0,18.0,46.0,6.0,20.0,20.0,5.000000


### Validate the changes made.

In [5]:
print("Original shape:", df.shape)
print("Cleaned shape:", df_cleaned.shape)
df_cleaned.info()

Original shape: (515, 17)
Cleaned shape: (493, 17)
<class 'pandas.core.frame.DataFrame'>
Index: 493 entries, 0 to 514
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   restaurant          493 non-null    object 
 1   item                493 non-null    object 
 2   calories            493 non-null    float64
 3   cal_fat             493 non-null    float64
 4   total_fat           493 non-null    float64
 5   sat_fat             493 non-null    float64
 6   trans_fat           493 non-null    float64
 7   cholesterol         493 non-null    float64
 8   sodium              493 non-null    float64
 9   total_carb          493 non-null    float64
 10  fiber               493 non-null    float64
 11  sugar               493 non-null    float64
 12  protein             493 non-null    float64
 13  vit_a               493 non-null    float64
 14  vit_c               493 non-null    float64
 15  calcium    

### Export for PowerBI analyzation.

In [6]:
cleaned_path = "C:/Users/Patrick Behnke/Desktop/Data Analytics/Projects/Project 2/Dataset/fastfood_cleaned.csv"
df_cleaned.to_csv(cleaned_path, index=False)
print("Cleaned dataset is saved to:", cleaned_path)

Cleaned dataset is saved to: C:/Users/Patrick Behnke/Desktop/Data Analytics/Projects/Project 2/Dataset/fastfood_cleaned.csv
